<a href="https://colab.research.google.com/github/Tariqameenkhan/LangChain-Function-Tool-Calling/blob/main/RAG_functioncalling_weather_API_Method_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.1 MB/s eta 0:00:00


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

In [11]:
llm = ChatGoogleGenerativeAI(api_key =GEMINI_API_KEY, model= "gemini-1.5-flash" , verbose=True)

In [19]:
from langchain_core.tools import tool
# from langchain.tools import Tool
import requests
from google.colab import userdata
OPENWEATHERMAP_API_KEY =userdata.get('WEATHER_API_KEY')

@tool
def get_current_weather(location: str) -> str:
    """Gets the current weather in a given location using OpenWeatherMap API.
    Input should be in the format 'city, country', e.g., 'London, UK'.
    """
    try:
        city, country = location.split(", ")
        base_url = "http://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": f"{city},{country}",
            "appid": OPENWEATHERMAP_API_KEY,
            "units": "metric"  # Use metric units for Celsius
        }
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes

        data = response.json()
        temperature = data["main"]["temp"]
        description = data["weather"][0]["description"]
        return f"The current weather in {city}, {country} is {description} and {temperature}°C."

    except (ValueError, KeyError, requests.exceptions.RequestException) as e:
        return f"Error getting weather information: {e}"

tools = [get_current_weather]


In [20]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    # verbose=True  # Enable debugging and calling output
)

In [21]:
# Example usage
# weather_info = agent.run("What's the weather like in pairs, france?")
weather_info = agent.run("What's the weather in karachi ?")

weather_info

'The current weather in Karachi, Pakistan is haze and 23.9°C.'

In [7]:
weather_info = agent.run("What's the weather in karachi ?")

weather_info



> Entering new AgentExecutor chain...
Thought: I need to use the `get_current_weather` function to get the current weather in Karachi.  I'll need to specify the country as well.  Let's assume it's Pakistan.
Action: get_current_weather
Action Input: Karachi, Pakistan
Observation: The current weather in Karachi, Pakistan is haze and 23.9°C.
Thought:Thought: I now know the final answer.
Final Answer: The current weather in Karachi, Pakistan is haze and 23.9°C.

> Finished chain.


'The current weather in Karachi, Pakistan is haze and 23.9°C.'

In [9]:
weather_info

'The current weather in Karachi, Pakistan is haze and 23.9°C.'

In [10]:
weather_info = agent.run("What's the weather like in pairs, france?")
weather_info



> Entering new AgentExecutor chain...
Thought: I need to get the current weather in Pairs, France.
Action: get_current_weather
Action Input: Paris, France
Observation: The current weather in Paris, France is mist and -0.99°C.
Thought:Question: What's the weather like in pairs, france?
Thought: I need to get the current weather in Pairs, France.
Action: get_current_weather
Action Input: Paris, France
Observation: The current weather in Paris, France is mist and -0.99°C.
Thought:Question: What's the weather like in Paris, France?
Thought: I need to get the current weather in Paris, France.
Action: get_current_weather
Action Input: Paris, France
Observation: The current weather in Paris, France is mist and -0.99°C.
Thought:Question: What's the weather like in pairs, france?
Thought: I need to get the current weather in Paris, France.  The question has a spelling error.
Action: get_current_weather
Action Input: Paris, France
Observation: The current weather in Paris, France is mist and -

'The current weather in Paris, France is mist and -0.99°C.'